In [83]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [84]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [85]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [86]:
#load dataset to pandas dataframe
news_dat=pd.read_csv('/content/drive/MyDrive/FakeNewsdataset/train.csv')

In [87]:
news_dat.shape

(20800, 5)

In [88]:
news_dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [89]:
news_dat.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [90]:
news_dat.describe

<bound method NDFrame.describe of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2    

In [91]:
news_dat.value_counts

<bound method DataFrame.value_counts of           id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   

In [92]:
news_dat.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [93]:
news_dat['author'].isnull().sum()/len(news_dat['author'])*100


9.408653846153845

In [94]:
1957/20800

0.09408653846153846

In [95]:
#replacing null values with empty string
news_dat=news_dat.fillna('')

In [96]:
#merging title and author name
news_dat['content']=news_dat['author']+"  "+news_dat['title']
print(news_dat)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [97]:

news_dat=news_dat.drop(['title','author'],axis=1)

In [98]:
print(news_dat)

          id                                               text  label  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...      1   
1          1  Ever get the feeling your life circles the rou...      0   
2          2  Why the Truth Might Get You Fired October 29, ...      1   
3          3  Videos 15 Civilians Killed In Single US Airstr...      1   
4          4  Print \nAn Iranian woman has been sentenced to...      1   
...      ...                                                ...    ...   
20795  20795  Rapper T. I. unloaded on black celebrities who...      0   
20796  20796  When the Green Bay Packers lost to the Washing...      0   
20797  20797  The Macy’s of today grew from the union of sev...      0   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...      1   
20799  20799    David Swanson is an author, activist, journa...      1   

                                                 content  
0      Darrell Lucus  House Dem Aide: We Didn’t Even

In [99]:
#separating the data & label
x=news_dat.drop('label',axis=1)
y=news_dat['label']

Stemming

In [100]:
port_stem=PorterStemmer()

In [101]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]'," ",content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content

In [102]:
news_dat['content']=news_dat['content'].apply(stemming)

In [103]:
print(news_dat['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [104]:
x=news_dat['content'].values
y=news_dat['label'].values

In [105]:
#convert textual data to numerical data
vectorizer=TfidfVectorizer()
vectorizer.fit(x)
x=vectorizer.transform(x)

In [106]:
print(x)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [107]:
print(y)

[1 0 1 ... 0 1 1]


Splitting the dataset to train and test

In [108]:
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.25,stratify=y,random_state=5)

In [109]:
print(xtrain)

  (0, 16221)	0.5383577704408081
  (0, 13698)	0.8427163882386541
  (1, 16412)	0.25483593608597704
  (1, 15074)	0.31281328084609544
  (1, 13499)	0.3369785742423729
  (1, 11811)	0.25395621308863864
  (1, 5749)	0.26443050334909735
  (1, 4616)	0.29513144141703695
  (1, 3425)	0.31281328084609544
  (1, 2519)	0.3904277736624691
  (1, 2188)	0.2176819973704265
  (1, 1766)	0.24603258107606793
  (1, 1691)	0.25366695737881945
  (1, 1522)	0.28210231691657134
  (2, 14316)	0.367036353034666
  (2, 13606)	0.2822032211254472
  (2, 11684)	0.22713145837023987
  (2, 9723)	0.367036353034666
  (2, 9487)	0.38986650190148925
  (2, 8836)	0.24228036471721967
  (2, 4807)	0.3485971168960417
  (2, 4474)	0.4181442125378955
  (2, 2004)	0.30377800593109405
  (3, 15079)	0.3936706176888281
  (3, 11807)	0.4304478974328588
  :	:
  (15596, 7910)	0.2814702038767552
  (15596, 6684)	0.3342825334761793
  (15596, 6001)	0.32932110462983233
  (15596, 1392)	0.20972208707733267
  (15597, 14430)	0.30432459178621923
  (15597, 13265)	0

Training the model:Logistic Regression

In [110]:
model=LogisticRegression()

In [112]:
model.fit(xtrain,ytrain)

LogisticRegression()

Evaluation--Accuracy Score

In [113]:
xtrain_prediction=model.predict(xtrain)
train_data_accuracy=accuracy_score(xtrain_prediction,ytrain)

In [114]:
print("Accuracy score of training data: ",train_data_accuracy)

Accuracy score of training data:  0.9871794871794872


In [115]:
xtest_prediction=model.predict(xtest)
test_data_accuracy=accuracy_score(xtest_prediction,ytest)

In [116]:
print("Accuracy score of testing data: ",test_data_accuracy)

Accuracy score of testing data:  0.9757692307692307


Predictive system

In [122]:
x_pred=xtest[2]
prediction=model.predict(x_pred)
print(prediction)

[0]


In [123]:
if(prediction[0]==0):
  print("Real news")
else:
  print("Fake news")

Real news


In [124]:
print(ytest[2])

0


Saving the trained model

In [125]:
import pickle

In [126]:
filename='FakeNewsTrainedModel.sav'
pickle.dump(model,open(filename,'wb'))

In [127]:
#loading the dumped file
loaded_model=pickle.load(open(filename,'rb'))

In [128]:
x_pred=xtest[2]
prediction=model.predict(x_pred)
print(prediction)

[0]
